# ML Pipeline
Applied commands that constitude an Machine Learning (ML) Pipeline.
### Libraries & Datasets
- Import Python libraries
- Load dataset from database 
- Define feature and target variables X and Y